### Provisioning KVM@TACC Resources

In this experiment we will be only having Block Store as the persistent Storage Resource which will be needed throughout the experiment as well we will be using a Kubernetes Cluster which will be running as part of the KVM@TACC on single node which will have all containers running

![image](../../images/kvm-setup.png)

In [ ]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local

Setting Openstack Environment to KVM@TACC

In [ ]:
export OS_REGION_NAME="KVM@TACC"
export OS_AUTH_URL="https://kvm.tacc.chameleoncloud.org:5000/v3"

In [ ]:
cd /work/code/terraform/kvm
sed -i 's/APPLICATION_CREDENTIAL_ID/YOUR_APPLICATION_CREDENTIAL_ID/g' clouds.yaml
sed -i 's/APPLICATION_CREDENTIAL_SECRET/YOUR_APPLICATION_CREDENTIAL_SECRET/g' clouds.yaml
cat clouds.yaml

In [ ]:
terraform init

In [ ]:
terraform plan

In [ ]:
terraform apply -auto-approve

### Installing Ansible

In [ ]:
PYTHONUSERBASE=/work/.local pip install --user ansible-core==2.16.9 ansible==9.8.0 jmespath==1.0.1 jsonschema==4.23.0 netaddr==1.3.0

### Install Pre K8S - Configuration

Update the Floating IP for SSH in the ansible Configuration file and verify it

In [ ]:
cd /work/code/ansible
sed -i 's/A.B.C.D/FLOATING_IP/g' ansible.cfg
cat ansible.cfg

In [ ]:
cd /work/code/ansible
ansible-playbook -i inventory.yml pre_k8s/pre_k8s_configure.yml

### Install K8S

In [ ]:
export ANSIBLE_CONFIG=/work/code/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles

In [ ]:
cd /work/code/ansible/k8s/kubespray
ansible-playbook -i ../inventory/mycluster --become --become-user=root ./cluster.yml

### Post Install Configuration

In [ ]:
cd /work/code/ansible
ansible-playbook -i inventory.yml post_k8s/post_k8s_configure.yml

### Installing Platform

Installing Platform will install MinIO and MLFlow

In [ ]:
cd /work/code/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_platform.yml

### Installing Monitoring Platform

Installing Prometheus and Grafana for Monitoring and Alerting

In [ ]:
cd /work/code/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_monitoring.yml

We will finally have the following services deployed
![image2](../../images/k8s-setup.png)